# **Реальный кейс от компании «IntelliVision»**

Вы можете выбрать в качестве проекта задачу, которая возникла у нашего партнёра — компании IntelliVision.

![](https://lms.skillfactory.ru/assets/courseware/v1/6c2526dacce0c2dd0d86e070c70a09c9/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_1.png)



[IntelliVision](https://www.intelli-vision.com/) разрабатывает ПО для обработки изображений и анализа видео. Компания проектирует и обучает нейронные сети, работает с потоковым распознаванием видео и звука.

Программное обеспечение от IntelliVision установлено на 5 млн камер в умных домах, промышленных помещениях и торговых центрах по всему миру, а их системы помощи водителю ADAS каждый день используют более 1.5 млн автовладельцев.

С 2018 компания является частью корпорации Nortek Security & Control, её главный офис находится в Сан-Хосе, штат Калифорния, США. Один из крупнейших центров разработки компании расположен в России — в Нижнем Новгороде.

## Постановка задачи

Один из ключевых проектов IntelliVision — Smart City/Transportation, система, обеспечивающая безопасность дорожного движения и более эффективную работу парковок. С помощью Smart City/Transportation можно контролировать сигналы светофоров и соблюдение ограничений скорости, определять виды транспортных средств, распознавать номерные знаки, считать автомобили и людей. Подробнее почитать о Smart City/Transportation и посмотреть демонстрацию её работы можно на официальном сайте [IntelliVision](https://www.intelli-vision.com/smart-city/).

![](https://lms.skillfactory.ru/assets/courseware/v1/7b127139095fb12476944463e2f9434d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_2.png)

![](https://lms.skillfactory.ru/assets/courseware/v1/0e18ae5ee2affbc6a41951960c1a9eba/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_3.png)

В основе всех перечисленных возможностей проекта лежит CV (Computer Vision, компьютерное зрение). Компания использует модели, для обучения которых применяются огромные размеченные датасеты с изображениями транспортных средств.

Система работает в режиме реального времени, поэтому данных с каждым днём всё больше. Алгоритм необходимо регулярно модернизировать, учитывая при этом множество факторов.

Для модификации и повышения эффективности системы Smart City/Transportation команде необходимо автоматизировать определение дополнительных параметров авто на изображении:

* тип автомобиля (кузова),
* ракурс снимка (вид сзади или спереди),
* цвет автомобиля и т. д.

Также необходимо автоматизировать поиск выбросов в данных (засветы и блики на изображениях; снимки, на которых отсутствуют автомобили и т. д.).

К сожалению, у компании нет комплексной модели, которая могла бы одновременно находить на изображении автомобиль и определять все нужные параметры. Её нужно построить, однако многокомпонентная разметка новых данных по всем этим параметрам — очень трудная задача, решение которой стоит больших денег.

Впрочем, у команды есть гипотеза, которая нуждается в проверке.

**Гипотеза**: разметку исходных данных можно эффективно провести с помощью методов кластеризации.

**В чём идея?**

Давайте будем использовать небольшой набор моделей свёрточных нейронных сетей, обученных на различных датасетах и решающих различные задачи от классификации изображений по цвету до классификации типов транспортных средств, пропустим нашу базу изображений через каждую модель, но возьмём не выходной результат модели, а только промежуточное представление признаков (дескриптор), полученное на свёрточных слоях сети.

Выполним такую операцию для всех изображений из набора данных, на основе полученных дескрипторов кластеризуем изображения, проинтерпретируем полученные кластеры и попробуем найти в них необходимую информацию.

**Что такое промежуточное представление (дескриптор) избражения?**

Мы пока не знакомы с нейронными сетями, а тем более с их подклассом — свёрточными нейронными сетями. Однако это никак не повлияет на реализацию проекта, так как в нём мы будем работать не с самими нейронными сетями, а только с заранее подготовленными дескрипторами. Для общего понимания задачи давайте попробуем разобраться, о чём здесь идёт речь.

Классическая свёрточная нейронная сеть, решающая задачу классификации, состоит из трёх частей:

* **Входной слой**, на который подаётся изображение. Мы должны определить класс этого изображения.
* **Свёрточные слои**, которые отвечают за выделение признаков на изображении. Выходом таких слоёв является некоторый тензор (набор матриц) размерности (h, w, c). В дальнейшем этот выход растягивается в одномерный вектор с помощью операции Flatten. Это вектор признаков, описывающих входное изображение. Он-то и называется промежуточным (латентным) представлением, или дескриптором.
* **Классификационные слои**, которые выполняют классификацию на основе признаков, выделенных свёрточными слоями. Выходом таких слоёв является вектор из вероятностей принадлежности к каждому из классов, заданных в данных. На основе этого вектора затем определяется класс входного изображения.

![](https://lms.skillfactory.ru/assets/courseware/v1/2de13bace4f930b2f16e2b7ce95f90bc/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_4.png)

Теперь, когда мы обсудили гипотезу, перейдём к постановке задачи.

У вас будет набор из 416 314 изображений транспортных средств различных типов, цветов и снятых с разных ракурсов.

![](https://lms.skillfactory.ru/assets/courseware/v1/8140338ea6c931173bd3000b387d734a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_5.png)



Команда IntelliVision уже обработала свой набор данных с помощью нескольких моделей глубокого обучения (свёрточных нейронных сетей) и получила четыре варианта вектора признаков (дескрипторов) для каждого изображения.

**Ваша задача:** используя готовые дескрипторы, разбить изображения на кластеры и проинтерпретировать каждый из них. Для всех вариантов дескрипторов нужно применить несколько алгоритмов кластеризации и сравнить полученные результаты. Сравнивать можно на основе метрик, визуализаций плотностей кластеров и по тому, насколько хорошо интерпретируются кластеры.

**Дополнительная подзадача:** найти выбросы среди изображений. Это могут быть изображения плохого качества, изображения с бликами или изображения, на которых нет транспортных средств и т. д.

**Бизнес-задача:** исследовать возможность применения алгоритмов кластеризации для разметки новых данных и поиска выбросов.

**Техническая задача для вас как для специалиста в Data Science:** построить модель кластеризации изображений на основе дескрипторов, выделяемых с помощью различных архитектур нейронных сетей, проинтерпретировать полученные результаты и выбрать модель или комбинацию моделей, которая выделяет наиболее пригодные для интерпретации признаки.

**Ваши основные цели:**

1. Для каждого типа дескрипторов необходимо:
    * выполнить предобработку дескрипторов;
    * произвести кластеризацию изображений на основе их дескрипторов, подобрав алгоритм и параметры кластеризации;
    * сделать визуализацию полученных кластеров в 2D- или 3D-пространстве;
    * проинтерпретировать полученные кластеры — в паре предложений сформулировать, какие изображения попали в каждый из кластеров.
2. Сравнить между собой полученные кластеризации для каждого типа дескрипторов (по метрикам, визуализации и результатам интерпретации).
3. Выполнить автоматизированный поиск выбросов среди изображений на основе дескрипторов.
4. Дополнительная задача (не оценивается): попробовать воспользоваться смесью дескрипторов, полученных различными моделями, и проинтерпретировать полученные результаты.

Артефактами вашей работы должны стать Jupyter Notebook с подробным исследованием, а также CSV-файл с лучшими результатами разделения на кластеры и выбросы.

**Примечание**. При выборе алгоритма кластеризации следует ориентироваться на внутренние метрики, а именно на индекс Калински — Харабаса (calinski_harabasz_score) и индекс Дэвиса — Болдина (davies_bouldin_score), а также на интерпретируемость кластеров и визуализацию.

### Почему этот кейс сложнее, чем кажется?
На первый взгляд кейс несложный: набор данных есть, дополнительные фичи делать не нужно — вызываем всевозможные алгоритмы кластеризации, смотрим на результат, и решение готово.

Однако один из главных вызовов этого кейса в том, что нужно обработать большой массив данных и провести множество экспериментов в ограниченный срок. Для оптимизации вычислений мы рекомендуем использовать библиотеку cuML, о которой мы рассказывали во введении к проекту.

## Данные и их описание

Исходные данные вы можете скачать по ссылке ниже.

**Осторожно!** Данные весят почти 30 Гб — предварительно убедитесь, что на вашем диске достаточно места.

[Скачать данные](https://drive.google.com/file/d/1vkQaj0Lr4Jwkumli7k9IzCtxFP1tIoXH/view)

**Примечание**. Также с этим датасетом можно работать на платформе Kaggle. Вы можете создать на его основе собственный ноутбук и работать в нём, не скачивая датасет.

[Датасет на Kaggle](https://www.kaggle.com/datasets/markhomeless/intellivision-case)

Исходная папка с данными имеет следующую структуру:
```
Intellivision_case  
├─descriptors  
    └─efficientnet-b7.pickle  
    └─osnet.pickle  
    └─vdc_color.pickle  
    └─vdc_type.pickle  
├─row_data  
└─veriwild.zip  
├─images_paths.csv
``` 

Давайте разберёмся в ней:

* В папке **descriptors** содержатся дескрипторы, полученные для каждого из изображений с помощью соответствующих нейронных сетей, в формате numpy-массивов, сохранённых в файлах **.pickle**:
    * **efficientnet-b7.pickle** — дескрипторы, выделенные моделью классификации с архитектурой EfficientNet версии 7. Эта модель является свёрточной нейронной сетью, предобученной на на датасете ImageNet, в котором содержатся изображения более 1000 различных классов. Эта модель при обучении не видела датасета veriwiId.
    * **osnet.pickle** — дескрипторы, выделенные моделью OSNet, обученной для детектирования людей, животных и машин. Модель не обучалась на исходном датасете veriwiId.
    * **vdc_color.pickle** — дескрипторы, выделенные моделью регрессии для определения цвета транспортных средств в формате RGB. Частично обучена на исходном датасете veriwild.
    * **vdc_type.pickle** — дескрипторы, выделенные моделью классификации транспортных средств по типу на десяти классах. Частично обучена на исходном датасете veriwild.

* В папке **row_data** содержится zip-архив с исходными изображениями автомобилей. Распакуйте его содержимое в папку **row_data**. Архив содержит десять папок с изображениями, пронумерованных от 1 до 10. Каждая папка содержит подпапки, обозначенные пятизначными цифрами, например 36191.

В каждой из таких подпапок содержатся фотографии одного конкретного автомобиля с разных ракурсов, снятые с помощью дорожных видеокамер.

![](https://lms.skillfactory.ru/assets/courseware/v1/76670a121a5a78845b57812ee36508b8/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_PJ_6_3_6.png)



* В файле **images_paths.csv** представлен список из полных путей до изображений. Он пригодится вам при анализе изображений, попавших в определённый кластер.

## Важные инсайты по данным, которые помогут вам в решении задачи:

* Каждая строка из numpy-матрицы, хранящейся в файле .pickle, является дескриптором соответствующего изображения, пути которого заданы в файле images_paths.csv.

* Размерности дескрипторов отличаются в зависимости от моделей, используемых для их обучения.

* В данных присутствуют выбросы: изображения крайне низкого качества, изображения с засветами/бликами и др.

# Предлагаемые этапы решения задачи и подсказки

Ниже дан примерный план решения задачи. Вы можете использовать его как основу для своего решения. Также мы привели несколько вспомогательных функций, которые могут помочь вам в процессе предобработки данных, а также при интерпретации кластеров.

Также мы подготовили для нас ноутбук-шаблон:

[Скачать ноутбук-шаблон](https://lms.skillfactory.ru/assets/courseware/v1/2480fbbcc9deea3df0794c8e42f120b7/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/PROJECT-6._%D0%9A%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D0%B9_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D0%BF%D0%BE%D1%80%D1%82%D0%BD%D1%8B%D1%85_%D1%81%D1%80%D0%B5%D0%B4%D1%81%D1%82%D0%B2.ipynb)